# Exploratory Analysis and PoC for creating a sentiment analysis app for the EPL Teams

In [9]:
import tweepy
import json
import pandas as pd
from datetime import datetime
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
# nltk.download('vader_lexicon') # Need this to run SIA()
# need to install pyarrow

In [67]:
df_query_list=pd.read_csv('query_list.csv')

In [68]:
df_query_list=df_query_list.head() #debug for first 5 rows

In [69]:
with open("C:/Users/William Jiang/Documents/credentials.json") as f:
    d = json.load(f)
    cred_json=d["twitter_api"]
    

# Authenticate to Twitter
auth = tweepy.OAuthHandler(cred_json["CONSUMER_KEY"], cred_json["CONSUMER_SECRET"])
auth.set_access_token(cred_json["ACCESS_TOKEN"], cred_json["ACCESS_TOKEN_SECRET"])

# Create API object
api = tweepy.API(auth)

# # Create a tweet
# api.update_status("Hello Tweepy2")

In [70]:
def polarity(row):
    #Does the sentiment analysis stuff
    sia = SIA()
    polarity=sia.polarity_scores(row)
    return polarity['compound']

In [71]:
import os
arr = os.listdir('output')

In [74]:
for index,row in df_query_list.iterrows():
    #initialize the search_id_list
    try:
        search_id_list=[]
        for file in arr:
            if ".parquet" in file and row['file_name'] in file and row['category'] in file:
                search_id=file.split('-')[2]
                search_id_list.append(search_id)
        
        if not search_id_list:
            last_since_id=None
        else:
            last_since_id=max(search_id_list)

        #Get Search Results
        search_results=api.search(q=row['search'], lang="en", count=15,since_id=last_since_id) #since_id neeeded here at some point
        rows_list=[]
        for tweet in search_results:
            dict1 = {}
            dict1['twitter_id']=tweet.id
            dict1['username']=tweet.user.name
            dict1['screenname']=tweet.user.screen_name
            dict1['text']=tweet.text
            dict1['created_at']=tweet.created_at
            dict1['retweet_count']=tweet.retweet_count
            dict1['location']=tweet.user.location
            dict1['followers']=tweet.user.followers_count
            dict1['friends']=tweet.user.friends_count
            dict1['search_query']=row['search']
            dict1['file_name']=row['file_name']
            dict1['group_name']=row['group_name']
            rows_list.append(dict1)

        #output to dataframe and then to snappy.parquet form
        df=pd.DataFrame(rows_list) 
        file_dir='output/'
        df['polarity']=df['text'].apply(polarity)
        to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
        max_twitter_id=max(df['twitter_id'])
        df.to_parquet(file_dir+row['category'] +'-'+ row['file_name'] +'-'+ str(max_twitter_id) + '-'+ to_csv_timestamp + '.snappy.parquet',compression='SNAPPY')
#         df.to_csv(file_dir+row['category'] +'-'+ row['file_name'] +'-'+ str(max_twitter_id) + '-'+ to_csv_timestamp + '.csv') #debugging
    except Exception as e:
        print(e)

In [ ]:
###Lets tweet this info#####

In [101]:
average_str='Average: ' + str(df['polarity'].mean().round(2))
median_str='Median: ' + str(df['polarity'].median().round(2))
std_str='Standard Deviation: ' + str(df['polarity'].std().round(2))
row1='Sentiment around: ' + df['file_name'][0]
row2='Search Query: ' + df['search_query'][0]
row3=average_str + ', ' + median_str + ', ' +  std_str

final_str=row1 + '\n'+ row2 + '\n' + row3
print(final_str)
# # Create a tweet
api.update_status(final_str)

Sentiment around: CrystalPalace
Search Query: #CrystalPalace OR #CPFC
Average: 0.35, Median: 0.4, Standard Deviation: 0.3


Status(_api=<tweepy.api.API object at 0x000001938ADCFA58>, _json={'created_at': 'Sat Jul 04 06:35:30 +0000 2020', 'id': 1279302799243661314, 'id_str': '1279302799243661314', 'text': 'Sentiment around: CrystalPalace\nSearch Query: #CrystalPalace OR #CPFC\nAverage: 0.35, Median: 0.4, Standard Deviation: 0.3', 'truncated': False, 'entities': {'hashtags': [{'text': 'CrystalPalace', 'indices': [46, 60]}, {'text': 'CPFC', 'indices': [64, 69]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://en.wikipedia.org/wiki/Uniform_Resource_Locator" rel="nofollow">bumbumapp</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 4073594774, 'id_str': '4073594774', 'name': 'Will', 'screen_name': 'iamwilliamj1', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_

In [90]:
#Most Positive text
df[df['polarity']==max(df['polarity'])]['text'].iloc[0]

'Let’s go! Great to be in contact and thanks @holmesdaleUSA for the intro! “Charleston Palace” has a great ring to i… https://t.co/OA6OrK3UoH'

In [91]:
#Most Negative text
df[df['polarity']==min(df['polarity'])]['text'].iloc[0]

"We've previewed today's #PremierLeague matches here - https://t.co/G2f2e23fCu\n\n#football #betting #tips #EPL… https://t.co/JJnef2CnIp"